<a href="https://colab.research.google.com/github/yxbxn/NLP_contest/blob/main/Back_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import pandas as pd
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tnrange
from urllib.request import urlopen
import re
import requests
import urllib.request
from tqdm import tqdm

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

In [ ]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

In [ ]:
driver=chrome_setting()

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
# 특정 파일 불러오기
import pandas as pd
train_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/test/train_preprocessed.csv")
test_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/test/test_preprocessed.csv")

In [ ]:
train_preprocessed.shape

AI_id 컬럼 부여
- 정수 자릿수 맞추기 : https://jlog1016.tistory.com/4

In [ ]:
train_id = []
for i in range(1,1000001):
    num = str(i).zfill(7)
    train_id.append("id_" + num)

In [ ]:
test_id = []
for i in range(1,100001):
    num = str(i).zfill(6)
    test_id.append("id_" + num)

In [ ]:
train_preprocessed["AI_id"] = train_id
test_preprocessed["AI_id"] = test_id

# **Data Augmentation**

In [ ]:
train_preprocessed["label"].value_counts().describe()

230개 미만인 label 제거

- except_label = train_preprocessed["label"].value_counts()[train_preprocessed["label"].value_counts() < 230].index
- except_label

In [ ]:
#mask = train_preprocessed['label'].isin(except_label)
#train_preprocessed = train_preprocessed[~mask].reset_index(drop=True)
#train_preprocessed.head()

In [ ]:
train_preprocessed["label"].value_counts().describe()

남아있는 label Q1 : 230
- label 개수가 230개 이하인 데이터 증강

In [ ]:
augment_label = train_preprocessed["label"].value_counts()[train_preprocessed["label"].value_counts() < 230].index
mask = train_preprocessed['label'].isin(augment_label)
augment_preprocessed = train_preprocessed[mask].reset_index(drop=True)
augment_preprocessed.head()

In [ ]:
# 증강시킬 데이터
augment_preprocessed

In [ ]:
import requests
import time
url = "https://translate.kakao.com/translator/translate.json"

def kakao_translate(query,from_lang,to_lang):

    headers = {
        "Referer": "https://translate.kakao.com/",
        "User-Agent": "Mozilla/5.0"}

    data = {
        "queryLanguage": from_lang,
        "resultLanguage": to_lang,
        "q": query}

    resp = requests.post(url, headers=headers, data=data)
    time.sleep(3)

    try:
        if resp.json()["result"]["status_code"] == 200:
            data = resp.json()['result']['output'][0][0]
    except:
        data = "None"
    return data

In [ ]:
txt = "니트 테이프 티리올 수지 중합 수지 배합"

if __name__ == "__main__":
    print(kakao_translate(txt,"kr","cn"))
    print(kakao_translate(txt,"kr","en"))
    print(kakao_translate(txt,"kr","jp"))

### 분할 시행

In [ ]:
tqdm.pandas()

augment_preprocessed["eng"] = ""
augment_preprocessed["chn"] = ""
augment_preprocessed["jpn"] = ""

In [ ]:
# 영어 1번
for i in tqdm(range(0,1000)):
    augment_preprocessed.loc[i,"eng"] = kakao_translate(augment_preprocessed.loc[i,"document"],"kr","en")

augment_preprocessed.to_csv("/content/MyDrive/MyDrive/test/augment.csv",encoding="utf-8-sig",index=False)

In [ ]:
# 영어 2번
for i in tqdm(range(1000,2000)):
    augment_preprocessed.loc[i,"eng"] = kakao_translate(augment_preprocessed.loc[i,"document"],"kr","en")

augment_preprocessed.to_csv("/content/MyDrive/MyDrive/test/augment.csv",encoding="utf-8-sig",index=False)

In [ ]:
augment_preprocessed[augment_preprocessed["eng"] == "None"]

In [ ]:
# 영어 3번
for i in tqdm(range(2000,3000)):
    augment_preprocessed.loc[i,"eng"] = kakao_translate(augment_preprocessed.loc[i,"document"],"kr","en")

augment_preprocessed.to_csv("/content/MyDrive/MyDrive/test/augment.csv",encoding="utf-8-sig",index=False)

In [ ]:
# 영어 4번
for i in tqdm(range(3000,4000)):
    augment_preprocessed.loc[i,"eng"] = kakao_translate(augment_preprocessed.loc[i,"document"],"kr","en")

augment_preprocessed.to_csv("/content/MyDrive/MyDrive/test/augment.csv",encoding="utf-8-sig",index=False)

In [ ]:
# 영어 5번
for i in tqdm(range(4000,len(augment_preprocessed))):
    augment_preprocessed.loc[i,"eng"] = kakao_translate(augment_preprocessed.loc[i,"document"],"kr","en")

augment_preprocessed.to_csv("/content/MyDrive/MyDrive/test/augment.csv",encoding="utf-8-sig",index=False)

중국어

In [ ]:
augment_preprocessed["chn"] = augment_preprocessed["document"].progress_apply(lambda x : kakao_translate(x,"kr","cn"))

In [ ]:
augment_preprocessed["jpn"] = augment_preprocessed["document"].progress_apply(lambda x : kakao_translate(x,"kr","jp"))

In [ ]:
augment_preprocessed[augment_preprocessed["eng"] == "None"]

## Papago Crawling 마지막..

In [ ]:
# Crawling

def kor_to_trans(text_data, trans_lang,start_index,final_index):
    trans_list = []
    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

    for i in tqdm(range(start_index,final_index)): 
    
        if (i!=0)&(i%99==0):
            time.sleep(2)
            print('{}th : '.format(i), backtrans)
    
        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.1)
            backtrans = element.text 

            if (backtrans=='')|(backtrans==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans = element.text 
                trans_list.append(backtrans)
            else:
                trans_list.append(backtrans)
        
        except:
            trans_list.append('')

    return trans_list

In [ ]:
augment_preprocessed.head(3)

In [ ]:
data_path = "/content/MyDrive/MyDrive/test/"

In [ ]:
trans_list = kor_to_trans(augment_preprocessed['document'], 'en',0,len(augment_preprocessed))
np.save(data_path+'kor_to_eng_train_0_4328.npy',trans_list)

할당

In [ ]:
augment_preprocessed["eng"] = trans_list

In [ ]:
trans_list = kor_to_trans(augment_preprocessed['document'], 'zn-CN',0,len(augment_preprocessed))
np.save(data_path+'kor_to_chn_train_0_4328.npy',trans_list)